## Fase 5 — EDA e Pré-processamento

Nesta fase damos início à **análise exploratória dos dados (EDA)**. O objetivo é garantir que o conjunto de dados brutos versionado esteja coerente, compreendendo estrutura, tipos de variáveis, dados faltantes e distribuição.  
Toda inspeção será feita **dentro do DevContainer**, garantindo rastreabilidade e reprodutibilidade total, alinhada ao versionamento DVC já configurado.


In [2]:
# ETAPA: LISTAR CONTEÚDO REAL DA PASTA data/raw/

import os

print("Diretório atual:", os.getcwd())
print("Conteúdo real de data/raw/:")
print(os.listdir("data/raw"))


Diretório atual: /workspace
Conteúdo real de data/raw/:
['.gitkeep', 'dummy.txt', 'dummy.txt.dvc', '.gitignore']
